# 📘 Sesión 11: PyTorch - Deep Learning

---

## 🎯 Objetivos

- Dominar tensores y autograd
- Construir redes neuronales
- Entrenar modelos con optimización
- Usar Datasets y DataLoaders

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import matplotlib.pyplot as plt

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

## 1. Tensores

In [ ]:
# Crear tensores
a = torch.tensor([1, 2, 3, 4])
b = torch.zeros(2, 3)
c = torch.ones(2, 3)
d = torch.randn(2, 3)  # Normal(0, 1)

print(f"Tensor a: {a}")
print(f"Shape: {a.shape}, Dtype: {a.dtype}")
print(f"\nRandom tensor:\n{d}")

In [ ]:
# Operaciones
x = torch.tensor([1.0, 2.0, 3.0])
y = torch.tensor([4.0, 5.0, 6.0])

print(f"Suma: {x + y}")
print(f"Producto: {x * y}")
print(f"Dot product: {torch.dot(x, y)}")
print(f"Matmul: {torch.mm(x.view(3, 1), y.view(1, 3))}")

# Reshape
z = torch.arange(12).reshape(3, 4)
print(f"\nReshaped:\n{z}")

In [ ]:
# Conversión NumPy <-> PyTorch
np_array = np.array([1, 2, 3])
tensor = torch.from_numpy(np_array)
back_to_numpy = tensor.numpy()

print(f"NumPy: {np_array}")
print(f"Tensor: {tensor}")
print(f"Back to NumPy: {back_to_numpy}")

## 2. Autograd

In [ ]:
# Gradientes automáticos
x = torch.tensor([2.0, 3.0], requires_grad=True)

# Forward pass
y = x ** 2
z = y.sum()

# Backward pass
z.backward()

print(f"x: {x}")
print(f"y = x²: {y}")
print(f"z = sum(y): {z}")
print(f"dz/dx: {x.grad}")  # Debería ser 2*x = [4, 6]

In [ ]:
# Ejemplo: descenso de gradiente manual
x = torch.tensor([5.0], requires_grad=True)
lr = 0.1

historial = []
for i in range(20):
    # f(x) = (x - 2)²
    y = (x - 2) ** 2
    historial.append((x.item(), y.item()))
    
    y.backward()
    
    with torch.no_grad():
        x -= lr * x.grad
    
    x.grad.zero_()

print(f"Óptimo encontrado: x = {x.item():.4f}")
plt.plot([h[0] for h in historial], [h[1] for h in historial], 'o-')
plt.xlabel('x'); plt.ylabel('f(x)'); plt.title('Convergencia')
plt.show()

## 3. Redes Neuronales con nn.Module

In [ ]:
# Definir red neuronal
class RedNeuronal(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        return x

model = RedNeuronal(10, 20, 2)
print(model)
print(f"\nParámetros: {sum(p.numel() for p in model.parameters())}")

In [ ]:
# Red con nn.Sequential
model_seq = nn.Sequential(
    nn.Linear(10, 64),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 2)
)

print(model_seq)

## 4. Entrenamiento Completo

In [ ]:
# Generar datos sintéticos
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)

# Convertir a tensores
X_tensor = torch.FloatTensor(X)
y_tensor = torch.LongTensor(y)

# Split
train_size = int(0.8 * len(X_tensor))
X_train, X_test = X_tensor[:train_size], X_tensor[train_size:]
y_train, y_test = y_tensor[:train_size], y_tensor[train_size:]

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

In [ ]:
# DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
# Modelo, loss, optimizer
model = nn.Sequential(
    nn.Linear(10, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 2)
)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 50
history = {'train_loss': [], 'test_acc': []}

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    
    for batch_X, batch_y in train_loader:
        # Forward
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    # Evaluación
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for batch_X, batch_y in test_loader:
            outputs = model(batch_X)
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
        
        accuracy = correct / total
    
    history['train_loss'].append(epoch_loss / len(train_loader))
    history['test_acc'].append(accuracy)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}, Acc: {accuracy:.4f}")

In [ ]:
# Visualizar entrenamiento
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].plot(history['train_loss'])
axes[0].set_title('Training Loss')
axes[0].set_xlabel('Epoch')

axes[1].plot(history['test_acc'])
axes[1].set_title('Test Accuracy')
axes[1].set_xlabel('Epoch')

plt.tight_layout()
plt.show()

## 5. GPU (si disponible)

In [ ]:
# Usar GPU si está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando: {device}")

# Mover modelo a GPU
# model = model.to(device)
# X_train = X_train.to(device)
# y_train = y_train.to(device)

---
## 🏋️ Ejercicios Resueltos

In [ ]:
# Ejercicio 1: Regresión con PyTorch
X_reg = torch.linspace(-5, 5, 100).reshape(-1, 1)
y_reg = 3 * X_reg + 2 + torch.randn_like(X_reg) * 0.5

model_reg = nn.Linear(1, 1)
optimizer = optim.SGD(model_reg.parameters(), lr=0.01)
criterion = nn.MSELoss()

for epoch in range(100):
    pred = model_reg(X_reg)
    loss = criterion(pred, y_reg)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f"Peso aprendido: {model_reg.weight.item():.2f} (real: 3)")
print(f"Bias aprendido: {model_reg.bias.item():.2f} (real: 2)")

---
## 📝 Ejercicios para Practicar

In [ ]:
# Ejercicio 1: Implementar red con Batch Normalization
# Tu código aquí

In [ ]:
# Ejercicio 2: Early stopping y guardar mejor modelo
# Tu código aquí

In [ ]:
# Ejercicio 3: Dataset personalizado para imágenes
# Tu código aquí

---
## 🎯 Resumen

- **Tensores**: torch.tensor(), operaciones, device
- **Autograd**: requires_grad, backward(), grad
- **nn.Module**: Definir arquitectura, forward()
- **Entrenamiento**: DataLoader, optimizer, criterion, training loop
- **GPU**: .to(device), cuda/cpu